***
# Applied Data Science Capstone Project
## Week 3 -  Segmenting and Clustering Neighborhoods in Toronto
## Notebook 2 - Add Geographical Coordenates to Canada Postal Codes
***

### Import Packages

In [9]:
import pandas as pd
import numpy as np

## Part 1 - Preparing Canada Postal Data

### Read the postal codes of Canada using Wikipedia page

In [10]:
# Read Canada Postal Code from Wikipedia table
#
can_post_URL  = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_can = pd.read_html(can_post_URL)[0]
df_can.rename(columns = {'Postcode':'PostalCode'}, inplace = True) 
print('Dimension of the Dataframe is', df_can.shape)
df_can.head()

Dimension of the Dataframe is (287, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Filter Data
- Ignore cells with a borough that is Not assigned
- Borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
- Combines neighborhoods that exist in one postal code area (Separated with a comma)

In [11]:
# Change "Not assigned" Neighbourhood with the Borough Value
# 
df_can_filter1 = df_can[(df_can['Borough'] != 'Not assigned')]
df_can_filter1.reset_index(drop=True, inplace=True)
print('The Dataframe Dimension after dropping Boroughs with "Not Assigned" values is',df_can_filter1.shape)
df_can_filter1.head()

The Dataframe Dimension after dropping Boroughs with "Not Assigned" values is (210, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [12]:
# Change "Not assigned" Neighbourhood with the Borough Value
#
df_can_filter2=df_can_filter1.copy()
df_can_filter2.loc[df_can_filter2['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_can_filter2['Borough']
df_can_filter2.reset_index(drop=True, inplace=True)
print('The following Dataframe does not contain "Not assigned" values')
df_can_filter2.head()

The following Dataframe does not contain "Not assigned" values


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [13]:
# Merge Rows with same PostalCode
#
df_can_filter2=df_can_filter2.sort_values(by ='Neighbourhood', ascending=[False] )
df_can_group = df_can_filter2.groupby(['PostalCode']).agg({'Borough':'first','Neighbourhood': ', '.join }) 
df_can_group = df_can_group.reset_index()
df_can_group.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Display the dimension (Shape) of the final table

In [14]:
# Display final Dataframe Dimension
#
table_dim = df_can_filter2.shape
print('The Final Filtered Table Dimension is',table_dim)

The Final Filtered Table Dimension is (210, 3)


## Part 2 - Obtain Latitude and Longitud Data

In [15]:
# Read Geo Data from the file provided in the instructions
#
geo_data='http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(geo_data,float_precision='round_trip')
df_geo = df_geo.rename(columns={'Postal Code': 'PostalCode'})  #Rename the Column to match the postal code dataframe
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the two dataframes

In [16]:
# Merge the two dataframes
#
df_geo_merge = pd.merge(df_can_group, df_geo)
df_geo_merge=df_geo_merge.sort_values(by ='PostalCode')
df_geo_merge

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"St. Phillips, Richview Gardens, Martin Grove G...",43.688905,-79.554724
101,M9V,Etobicoke,"Thistletown, South Steeles, Silverstone, Mount...",43.739416,-79.588437


### Display the dimension (Shape) of the final Geo table

In [10]:
# Display the Merge the two dataframes
#
table_geo_dim = df_geo_merge.shape
print('The Final Filtered Table Dimension is',table_geo_dim)

The Final Filtered Table Dimension is (103, 5)
